# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect single and multi person poses on the MPII Dataset


## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install OpenPose on Google Colab 


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`




# From Google Drive - MPII DATASET

In [ ]:
#Extracting Images onto Drive (need to run only once)
# Unzips the original MPII dataset (image.zip) and stores it in another directory in the Google Drive.

!unzip -u "/content/drive/MyDrive/CIS 581 Final Project/Project/images.zip" -d "/content/drive/MyDrive/CIS 581 Final Project/Project/Full MPII Dataset"

In [ ]:
#Storing as Images with UI skeletons overlaid

colab_image_path = '/content/drive/MyDrive/CIS 581 Final Project/Project/Full MPII Dataset'
colab_openpose_image_path = '/content/drive/MyDrive/CIS 581 Final Project/Project/Full_MPII_Images_Output'

!cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 12 --image_dir '{colab_image_path}' --display 0 --write_images '{colab_openpose_image_path}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25

In [ ]:
#Storing as .json files

colab_image_path = '/content/drive/MyDrive/CIS 581 Final Project/Project/Full MPII Dataset'
colab_openpose_JSON_path = '/content/drive/MyDrive/CIS 581 Final Project/Project/Full_MPII_JSON_Output'

!cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 1 --image_dir '{colab_image_path}' --display 0 --write_json '{colab_openpose_JSON_path}' --render_pose 0 # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25

# Parsing JSON Files

In [ ]:
import json
import glob 
import numpy as np

full_dataset_path = "drive/MyDrive/CIS 581 Final Project/Project/Output_MPII_Full/*.json"
files = glob.glob("/content/" + full_dataset_path)

n_joints = 25

dataset = []
dataset2 = []
c = 0

#for each .json file
for filename in files:
    c+=1
    with open(filename, 'r') as json_data:
        python_dict = json.load(json_data)

        #for each person detected in the image 
        for candidates in python_dict['people']:
            
            #extracting x,y coordinates
            pose2dx = np.array(candidates['pose_keypoints_2d'][::3])
            pose2dy = np.array(candidates['pose_keypoints_2d'][1::3])
            pose2d = np.vstack((pose2dx, pose2dy)).T
            dataset.append(pose2d)  #contains 2D pose

            #parsing filename
            head, tail = os.path.split(filename)
            img_number = tail.split('_')
            img_number = img_number[0] + ".jpg"
            dataset2.append(img_number)  #contains corresponding image numbers for visualization
            
        if(c == 20000):
          break

dataset = np.array(dataset)
dataset2 = np.array(dataset2)

In [ ]:
#saving as .npy files 
test_file_path = "/content/drive/MyDrive/CIS 581 Final Project/Project/dataset.npy"
np.save(test_file_path, dataset)

test_file_path = "/content/drive/MyDrive/CIS 581 Final Project/Project/dataset2.npy"
np.save(test_file_path, dataset2)